In [117]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import six
import tensorflow as tf

from edward.models import (
    Dirichlet, Categorical, Empirical, ParamMixture)

plt.style.use('ggplot')

In [118]:
def build_toy_dataset(D, S, V, K):
    """
    Builds a toy dataset of D individuals.
    
    Args:
        D - Number of individuals
        S - Number of data sources
        V - List of data source vocabulary sizes
        K - Number of topics
        
    Returns:
        w - List of lists of 1D NumPy arrays containing
            tokens for each individual and data source
        theta - 2D NumPy array containing document
                topic distributions
        phi - 2D NumPy array containing topic
               token distributions
    """
    # Draw number of tokens for each individual
    N = [None] * D
    for d in range(D):
        N[d] = np.random.randint(low=2,
                              high=10,
                              size=S)
    
    # Draw topic distributions for each individual
    theta = np.random.dirichlet(alpha=np.ones(K) * 0.1, 
                                size=D)

    # Create topics for each data source (non-overlapping)
    phi = [None] * S
    for s in range(S):
        phi_values = np.array([K * 1 / V[s]] * int(V[s] / K) + [0.0] * int(V[s] - V[s] / K))
        phi[s] = np.zeros([K, V[s]])
        for k in range(K):
            phi[s][k, :] = np.roll(phi_values, int(k * V[s] / K))
    
    # Draw tokens for each document
    w = [[None] * S for d in range(D)]
    z = [[None] * S for d in range(D)] 
    for d in range(D):
        for s in range(S):
            # Draw token topic assignments
            z[d][s] = np.array([np.random.choice(range(K), size=N[d][s], p=theta[d, :])])[0]
            # Draw tokens
            w[d][s] = np.zeros(N[d][s])

            for n in range(N[d][s]):
                w[d][s][n] = np.random.choice(range(V[s]), size=1, p=phi[s][z[d][s][n], :]) 

    return N, w, z, theta, phi

In [119]:
###############
# DATA
###############

# Get toy dataset
D = 3
V = [10, 20, 30]
K = 2
S = len(V)
N_train, w_train, z_train, theta_train, phi_train = build_toy_dataset(D, S, V, K)


In [120]:
###############
# MODEL
###############

alpha = tf.ones(K) * 0.01

beta = [None] * S
phi = [None] * S
for s in range(S):
    beta[s] = tf.ones(V[s]) * 0.01
    phi[s] = Dirichlet(concentration=beta[s], 
                     sample_shape=K)

theta = [None] * D
w = [[None] * S for d in range(D)]
z = [[None] * S for d in range(D)]
for d in range(D):
    theta[d] = Dirichlet(concentration=alpha)
    
    for s in range(S):
        w[d][s] = ParamMixture(mixing_weights=theta[d], 
                            component_params={'probs': phi[s]},
                            component_dist=Categorical,
                            sample_shape=len(w_train[d][s]),
                            validate_args=True)
        
        z[d][s] = w[d][s].cat


In [122]:
####################
#INFERENCE
####################

# Data vars
data_dict={}
for d in range(D):
    for s in range(S):
        data_dict[w[d][s]] = w_train[d][s]


# Latent vars
latent_vars_dict = {}

T = 500 # number of samples
qphi = [None] * S
for s in range(S):
    qphi[s] = Empirical(tf.Variable(tf.zeros([T, K, V[s]])))
    latent_vars_dict[phi[s]] = qphi[s]

qtheta = [None] * D
qz = [[None] * S for d in range(D)]
for d in range(D):
    qtheta[d]= Empirical(tf.Variable(tf.ones([T, K]) / K))
    latent_vars_dict[theta[d]] = qtheta[d]
    
    for s in range(S):
        N = len(w_train[d][s])

        qz[d][s] = Empirical(tf.Variable(tf.zeros([T, N], dtype=tf.int32)))
        latent_vars_dict[z[d][s]] = qz[d][s]
    

# Proposal vars
proposal_vars_dict = {}

phi_cond = [None] * S
for s in range(S):
    phi_cond[s] = ed.complete_conditional(phi[s])
    proposal_vars_dict[phi[s]] = phi_cond[s]

theta_cond = [None] * D
z_cond = [[None] * S for d in range(D)]
for d in range(D):
    print('Building proposals for doc {} of {}'.format(d + 1, D))
          
    theta_cond[d] = ed.complete_conditional(theta[d])
    proposal_vars_dict[theta[d]] = theta_cond[d]
    
    for s in range(S):
        z_cond[d][s] = ed.complete_conditional(z[d][s])
        proposal_vars_dict[z[d][s]] = z_cond[d][s]
    



Building proposals for doc 1 of 3
Building proposals for doc 2 of 3
Building proposals for doc 3 of 3


In [123]:
# Inference procedure w/Gibbs sampling
inference = ed.Gibbs(latent_vars=latent_vars_dict,
                     proposal_vars=proposal_vars_dict,
                     data=data_dict)

inference.initialize(n_iter=T, n_print=10, logdir='log')

tf.global_variables_initializer().run()

for n in range(inference.n_iter):
    info_dict = inference.update()
    inference.print_progress(info_dict)
    
inference.finalize()

500/500 [100%] ██████████████████████████████ Elapsed: 252s | Acceptance Rate: 1.000


In [6]:
# def build_toy_dataset(D, V, N, K):

#     alpha = np.ones(K) * 0.2
#     eta = np.ones(V) * 0.2

#     theta = np.random.dirichlet(alpha, size=D)
#     beta = np.random.dirichlet(eta, size=K)

#     Z = np.array([np.random.choice(range(K), size=N, p=theta_p) for theta_p in theta])

#     W = np.zeros([D, N])
#     for d in range(D):
#         for n in range(N):
#             z = Z[d, n]
#             W[d, n] = np.random.choice(range(V), size=1, p=beta[z]) 

#     return W


# def build_toy_dataset_fixed(D, N):
    
#     K = 2
#     V = 50

#     alpha = np.ones(K) * 0.9
#     theta = np.random.dirichlet(alpha, size=D)
    
# #     eta_values = np.array([0.2] * int(V / K) + [0.0] * int(V - V / K))
# #     beta = np.zeros([K, V])
# #     for k in range(K):
# #         eta = np.roll(eta_values, int(k * V / K))
# #         beta[k, :] = np.random.dirichlet(eta)

#     beta_values = np.array([K * 1 / V] * int(V / K) + [0.0] * int(V - V / K))
#     beta = np.zeros([K, V])
#     for k in range(K):
#         beta[k, :] = np.roll(beta_values, int(k * V / K))
        
#     beta = np.array([beta[0, :], beta[-1, :]])

#     Z = np.array([np.random.choice(range(K), size=N, p=theta_p) for theta_p in theta])

#     W = np.zeros([D, N])
#     for d in range(D):
#         for n in range(N):
#             z = Z[d, n]
#             W[d, n] = np.random.choice(range(V), size=1, p=beta[z, :]) 

#     return W, K, V, beta

# def build_toy_dataset_fixed2(D, N):
    
#     K = 2
#     V = 50

#     alpha = np.ones(K) * 1.0
#     theta = np.random.dirichlet(alpha, size=D)

#     beta_values = np.array([5 * 1 / V] * int(V / 5) + [0.0] * int(V - V / 5))
#     beta = np.zeros([K, V])

#     beta[0, :] = beta_values
#     beta[1, :] = np.roll(beta_values, int(3 * V / K))

#     Z = np.array([np.random.choice(range(K), size=N, p=theta_p) for theta_p in theta])

#     W = np.zeros([D, N])
#     for d in range(D):
#         for n in range(N):
#             z = Z[d, n]
#             W[d, n] = np.random.choice(range(V), size=1, p=beta[z, :]) 

#     return W, K, V, Z, beta, theta


# def build_toy_dataset_fixed3(D, N):
    
#     K = 3
#     V = 30

#     alpha = np.ones(K) * 1.0
#     theta = np.random.dirichlet(alpha, size=D)

#     beta = np.zeros([K, V])
    
#     beta[0, 0] = 0.85
#     beta[0, 1] = 0.15
#     beta[1, 10] = 0.15
#     beta[1, 11] = 0.85
#     beta[2, 20] = 0.95
#     beta[2, 21] = 0.05

#     Z = np.array([np.random.choice(range(K), size=N, p=theta_p) for theta_p in theta])

#     W = np.zeros([D, N])
#     for d in range(D):
#         for n in range(N):
#             z = Z[d, n]
#             W[d, n] = np.random.choice(range(V), size=1, p=beta[z, :]) 

#     return W, K, V, Z, beta, theta


# def build_toy_dataset_variable(N, V):

#     D = len(N)
#     K = 3
    
#     alpha = np.ones(K) * 0.01
#     theta = np.random.dirichlet(alpha, size=D)

#     beta = np.zeros([K, V])
    
#     beta[0, 0] = 0.85
#     beta[0, 1] = 0.15
#     beta[1, 10] = 0.15
#     beta[1, 11] = 0.85
#     beta[2, 20] = 0.95
#     beta[2, 21] = 0.05
    
#     w, z = [0] * D, [0] * D
#     for d in range(D):
#         w[d] = np.zeros(N[d])
#         z[d] = np.array([np.random.choice(range(K), size=N[d], p=theta[d, :])])[0]
        
#         for n in range(N[d]):
#             w[d][n] = np.random.choice(range(V), size=1, p=beta[z[d][n], :]) 

#     return K, w, z, beta, theta
